In [2]:
import pandas as pd
import json
from pandas import json_normalize

In [ ]:
path = 'path/to/recipes/dataset.csv'

In [4]:
df = pd.read_csv(path)

In [5]:
df.head()

,recipe_name,source,url,servings,calories,total_weight_g,image_url,diet_labels,health_labels,cautions,cuisine_type,meal_type,dish_type,ingredient_lines,ingredients,total_nutrients,daily_values,digest
0,Classic Cabbage Slaw with Grandmother Shinn's ...,Food Network,https://www.foodnetwork.com/recipes/classic-ca...,6.0,511.283250,1239.311259,https://datahive-prod-dataset-products.s3.eu-c...,"[""Balanced""]","[""Vegetarian"",""Gluten-Free"",""Peanut-Free"",""Tre...",[],"[""american""]","[""lunch/dinner""]","[""salad""]","[""1 tablespoon kosher salt"",""2 cups water"",""4 ...","[{""food"":""kosher salt"",""text"":""1 tablespoon ko...","{""K"":{""unit"":""mg"",""label"":""Potassium"",""quantit...","{""K"":{""unit"":""%"",""label"":""Potassium"",""quantity...","[{""sub"":[{""tag"":""FASAT"",""unit"":""g"",""daily"":28...."
1,Black Bean Soup,Cookstr,http://www.cookstr.com/recipes/black-bean-soup...,8.0,1850.998990,3339.583230,https://datahive-prod-dataset-products.s3.eu-c...,"[""High-Fiber""]","[""Dairy-Free"",""Gluten-Free"",""Egg-Free"",""Peanut...","[""Sulfites""]","[""american""]","[""lunch/dinner""]","[""soup""]","[""1 pound fully cooked bone-in ham steak"",""1 t...","[{""food"":""ham steak"",""text"":""1 pound fully coo...","{""K"":{""unit"":""mg"",""label"":""Potassium"",""quantit...","{""K"":{""unit"":""%"",""label"":""Potassium"",""quantity...","[{""sub"":[{""tag"":""FASAT"",""unit"":""g"",""daily"":77...."
2,Eat for Eight Bucks: Tofu with Tomatoes and Ci...,Serious Eats,http://www.seriouseats.com/recipes/2010/06/eat...,4.0,1643.758565,1453.960928,https://datahive-prod-dataset-products.s3.eu-c...,"[""High-Fiber"",""Low-Carb""]","[""Vegan"",""Vegetarian"",""Dairy-Free"",""Egg-Free"",...","[""Gluten"",""Wheat"",""Sulfites""]","[""asian""]","[""lunch/dinner""]","[""main course""]","[""1 pound medium to firm tofu, cut into 1-inch...","[{""food"":""firm tofu"",""text"":""1 pound medium to...","{""K"":{""unit"":""mg"",""label"":""Potassium"",""quantit...","{""K"":{""unit"":""%"",""label"":""Potassium"",""quantity...","[{""sub"":[{""tag"":""FASAT"",""unit"":""g"",""daily"":54...."
3,Fried Chicken Banh Mi,Food Network,https://www.foodnetwork.com/recipes/fried-chic...,4.0,8471.182075,2547.239375,https://datahive-prod-dataset-products.s3.eu-c...,"[""High-Fiber""]","[""Peanut-Free"",""Tree-Nut-Free"",""Fish-Free"",""Sh...","[""Gluten"",""Wheat"",""Sulfites""]","[""south east asian""]","[""lunch/dinner""]","[""sandwiches""]","[""Neutral oil, for frying"",""1/2 cup unseasoned...","[{""food"":""oil"",""text"":""Neutral oil, for frying...","{""K"":{""unit"":""mg"",""label"":""Potassium"",""quantit...","{""K"":{""unit"":""%"",""label"":""Potassium"",""quantity...","[{""sub"":[{""tag"":""FASAT"",""unit"":""g"",""daily"":0,""..."
4,The Macaron Frappé,French Revolution Food,http://www.frenchrevolutionfood.com/2013/07/le...,1.0,276.243903,438.762998,https://datahive-prod-dataset-products.s3.eu-c...,[],"[""Vegetarian"",""Peanut-Free"",""Tree-Nut-Free"",""S...","[""Sulfites""]","[""french""]","[""snack""]","[""desserts""]","[""1 macaron (about ½ ounce), in any flavor you...","[{""food"":""macaron"",""text"":""1 macaron (about ½ ...","{""K"":{""unit"":""mg"",""label"":""Potassium"",""quantit...","{""K"":{""unit"":""%"",""label"":""Potassium"",""quantity...","[{""sub"":[{""tag"":""FASAT"",""unit"":""g"",""daily"":47...."


In [6]:
df["calorie_count"] = (df["calories"]//df["servings"])

In [7]:
df["serving_size_gms"] = df["total_weight_g"]//df["servings"]

In [8]:
df = df.drop( ["source", "url", "calories", "total_weight_g","image_url", "ingredient_lines", "digest"],axis=1)


In [10]:
import json

df["nutrients_dict"] = df["total_nutrients"].apply(json.loads)


In [12]:
from pandas import json_normalize

nutrients_expanded = json_normalize(df["nutrients_dict"])


In [13]:
nutrients_expanded = nutrients_expanded.rename(
    columns=lambda x: x.replace(".quantity", "")
)


In [14]:
cols_to_keep = [c for c in nutrients_expanded.columns if not c.endswith(("label", "unit"))]
nutrients_expanded = nutrients_expanded[cols_to_keep]


In [15]:
df = pd.concat([df, nutrients_expanded], axis=1)


In [17]:
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
df.head()


,recipe_name,servings,diet_labels,health_labels,cautions,cuisine_type,meal_type,dish_type,ingredients,total_nutrients,daily_values,calorie_count,serving_size_gms,nutrients_dict,K,P,CA,FE,MG,NA,ZN,FAT,NIA,FAMS,FAPU,RIBF,THIA,VITC,VITD,CHOLE,FASAT,FATRN,FIBTG,FOLAC,FOLFD,SUGAR,VITK1,WATER,CHOCDF,FOLDFE,PROCNT,TOCPHA,VITB12,VITB6A,VITA_RAE,CHOCDF.net,ENERC_KCAL,SUGAR.added
0,Classic Cabbage Slaw with Grandmother Shinn's ...,6.0,"[""Balanced""]","[""Vegetarian"",""Gluten-Free"",""Peanut-Free"",""Tre...",[],"[""american""]","[""lunch/dinner""]","[""salad""]","[{""food"":""kosher salt"",""text"":""1 tablespoon ko...","{""K"":{""unit"":""mg"",""label"":""Potassium"",""quantit...","{""K"":{""unit"":""%"",""label"":""Potassium"",""quantity...",85.0,206.0,"{'K': {'unit': 'mg', 'label': 'Potassium', 'qu...",1344.937283,378.238500,316.216893,4.986130,93.095613,2866.738895,2.631570,15.262045,2.399385,5.073339,2.436278,0.750850,0.346041,124.065000,2.000000,386.160000,5.740132,0.0,12.633475,0.0,241.737750,55.653280,385.811000,1111.919491,74.524150,241.737750,19.597500,2.835580,0.940400,0.746793,1260.135250,61.890675,511.283250,0.0
1,Black Bean Soup,8.0,"[""High-Fiber""]","[""Dairy-Free"",""Gluten-Free"",""Egg-Free"",""Peanut...","[""Sulfites""]","[""american""]","[""lunch/dinner""]","[""soup""]","[{""food"":""ham steak"",""text"":""1 pound fully coo...","{""K"":{""unit"":""mg"",""label"":""Potassium"",""quantit...","{""K"":{""unit"":""%"",""label"":""Potassium"",""quantity...",231.0,417.0,"{'K': {'unit': 'mg', 'label': 'Potassium', 'qu...",5951.657618,2361.677645,572.269647,23.629833,712.189757,12553.539068,20.479998,55.539721,22.703771,27.547140,7.578004,1.925726,4.132669,79.460838,4.082331,317.514659,15.555026,0.0,56.502798,0.0,1271.057411,11.406618,1144.806458,2871.681702,158.914882,1271.057411,181.811980,12.803019,3.991613,3.054568,1519.692579,102.412084,1850.998990,NaN
2,Eat for Eight Bucks: Tofu with Tomatoes and Ci...,4.0,"[""High-Fiber"",""Low-Carb""]","[""Vegan"",""Vegetarian"",""Dairy-Free"",""Egg-Free"",...","[""Gluten"",""Wheat"",""Sulfites""]","[""asian""]","[""lunch/dinner""]","[""main course""]","[{""food"":""firm tofu"",""text"":""1 pound medium to...","{""K"":{""unit"":""mg"",""label"":""Potassium"",""quantit...","{""K"":{""unit"":""%"",""label"":""Potassium"",""quantity...",410.0,363.0,"{'K': {'unit': 'mg', 'label': 'Potassium', 'qu...",3055.594090,1334.753701,3238.233026,19.513043,413.247690,2006.309099,10.884267,102.616128,15.456769,46.301237,40.946272,1.398109,1.630974,91.623990,0.293323,0.000000,10.899882,0.0,24.691537,84.0,318.857964,0.000000,0.000000,1089.618589,103.412350,461.357964,101.236576,0.000000,0.058665,1.305936,583.410517,78.720812,1643.758565,NaN
3,Fried Chicken Banh Mi,4.0,"[""High-Fiber""]","[""Peanut-Free"",""Tree-Nut-Free"",""Fish-Free"",""Sh...","[""Gluten"",""Wheat"",""Sulfites""]","[""south east asian""]","[""lunch/dinner""]","[""sandwiches""]","[{""food"":""oil"",""text"":""Neutral oil, for frying...","{""K"":{""unit"":""mg"",""label"":""Potassium"",""quantit...","{""K"":{""unit"":""%"",""label"":""Potassium"",""quantity...",2117.0,636.0,"{'K': {'unit': 'mg', 'label': 'Potassium', 'qu...",0.000000,0.000000,0.000000,0.000000,0.000000,7959.661000,0.000000,559.329245,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,833.645000,0.000000,0.0,32.722800,0.0,0.000000,0.000000,0.000000,0.000000,662.326780,0.000000,203.312290,0.000000,0.000000,0.000000,0.000000,629.603980,8471.182075,0.0
4,The Macaron Frappé,1.0,[],"[""Vegetarian"",""Peanut-Free"",""Tree-Nut-Free"",""S...","[""Sulfites""]","[""french""]","[""snack""]","[""desserts""]","[{""food"":""macaron"",""text"":""1 macaron (about ½ ...","{""K"":{""unit"":""mg"",""label"":""Potassium"",""quantit...","{""K"":{""unit"":""%"",""label"":""Potassium"",""quantity...",276.0,438.0,"{'K': {'unit': 'mg', 'label': 'Potassium', 'qu...",309.814957,181.418838,230.146385,0.212233,27.916563,148.884705,1.033935,14.428496,0.216324,3.179054,0.651036,0.373085,0

In [18]:
# Mapping: acronym -> (Full Name, Unit)
nutrient_map = {
    "ENERC_KCAL": ("Energy", "kcal"),
    "FAT": ("Total Fat", "g"),
    "FASAT": ("Saturated Fat", "g"),
    "FAMS": ("Monounsaturated Fat", "g"),
    "FAPU": ("Polyunsaturated Fat", "g"),
    "CHOCDF": ("Carbohydrates", "g"),
    "CHOCDF.net": ("Net Carbohydrates", "g"),
    "FIBTG": ("Dietary Fiber", "g"),
    "SUGAR": ("Total Sugars", "g"),
    "SUGAR.added": ("Added Sugars", "g"),
    "PROCNT": ("Protein", "g"),
    "CHOLE": ("Cholesterol", "mg"),
    "NA": ("Sodium", "mg"),
    "K": ("Potassium", "mg"),
    "CA": ("Calcium", "mg"),
    "MG": ("Magnesium", "mg"),
    "FE": ("Iron", "mg"),
    "ZN": ("Zinc", "mg"),
    "P": ("Phosphorus", "mg"),
    "VITA_RAE": ("Vitamin A (RAE)", "µg"),
    "VITC": ("Vitamin C", "mg"),
    "TOCPHA": ("Vitamin E (alpha-tocopherol)", "mg"),
    "VITK1": ("Vitamin K", "µg"),
    "THIA": ("Thiamin (B1)", "mg"),
    "RIBF": ("Riboflavin (B2)", "mg"),
    "NIA": ("Niacin (B3)", "mg"),
    "VITB6A": ("Vitamin B6", "mg"),
    "FOLDFE": ("Folate, DFE", "µg"),
    "FOLFD": ("Folate, food", "µg"),
    "FOLAC": ("Folic Acid", "µg"),
    "VITB12": ("Vitamin B12", "µg"),
    "VITD": ("Vitamin D", "µg"),
    "VITB5": ("Pantothenic Acid", "mg"),
    "VITB7": ("Biotin", "µg"),
    "WATER": ("Water", "g"),
    "ASH": ("Ash (Total Minerals)", "g")
}


In [19]:
def rename_nutrient(col):
    if col in nutrient_map:
        name, unit = nutrient_map[col]
        if unit == "g":
            suffix = "_gms"
        elif unit == "mg":
            suffix = "_mgs"
        elif unit == "µg":
            suffix = "_mcgs"
        else:
            suffix = ""
        return f"{name}{suffix}"
    else:
        return col

df.rename(columns=rename_nutrient, inplace=True)


In [22]:
df = df.drop("daily_values", axis=1)


In [24]:
df = df.drop("Energy", axis=1)


In [ ]:

df["servings"] = df["servings"].replace(0, pd.NA).astype(float)


In [27]:
nutrient_cols = [
    'Potassium_mgs', 'Phosphorus_mgs', 'Calcium_mgs', 'Iron_mgs', 'Magnesium_mgs', 
    'Sodium_mgs', 'Zinc_mgs', 'Total Fat_gms', 'Niacin (B3)_mgs', 'Monounsaturated Fat_gms', 
    'Polyunsaturated Fat_gms', 'Riboflavin (B2)_mgs', 'Thiamin (B1)_mgs', 'Vitamin C_mgs', 
    'Vitamin D_mcgs', 'Cholesterol_mgs', 'Saturated Fat_gms', 'FATRN', 'Dietary Fiber_gms', 
    'Folic Acid_mcgs', 'Folate, food_mcgs', 'Total Sugars_gms', 'Vitamin K_mcgs', 'Water_gms', 
    'Carbohydrates_gms', 'Folate, DFE_mcgs', 'Protein_gms', 'Vitamin E (alpha-tocopherol)_mgs', 
    'Vitamin B12_mcgs', 'Vitamin B6_mgs', 'Vitamin A (RAE)_mcgs', 'Net Carbohydrates_gms'
]


In [ ]:

for col in nutrient_cols:
    df[col + "_per_serving"] = df[col] / df["servings"]


df[[c + "_per_serving" for c in nutrient_cols]] = df[[c + "_per_serving" for c in nutrient_cols]].fillna(0)


In [30]:
df = df.drop("FATRN", axis=1)


In [32]:
df = df.drop("total_nutrients", axis=1)


In [33]:
df = df.drop("nutrients_dict", axis=1)


In [35]:
df= df.drop("servings", axis = 1)

In [ ]:

def format_labels(label_list):
    if not label_list:  
        return None
    label_list = [label.replace("-", " ") for label in label_list]  
    return ", ".join(label_list)

df["diet_labels_clean"] = df["diet_labels"].apply(format_labels)


In [38]:
df.head()

,recipe_name,diet_labels,health_labels,cautions,cuisine_type,meal_type,dish_type,ingredients,calorie_count,serving_size_gms,Potassium_mgs,Phosphorus_mgs,Calcium_mgs,Iron_mgs,Magnesium_mgs,Sodium_mgs,Zinc_mgs,Total Fat_gms,Niacin (B3)_mgs,Monounsaturated Fat_gms,Polyunsaturated Fat_gms,Riboflavin (B2)_mgs,Thiamin (B1)_mgs,Vitamin C_mgs,Vitamin D_mcgs,Cholesterol_mgs,Saturated Fat_gms,Dietary Fiber_gms,Folic Acid_mcgs,"Folate, food_mcgs",Total Sugars_gms,Vitamin K_mcgs,Water_gms,Carbohydrates_gms,"Folate, DFE_mcgs",Protein_gms,Vitamin E (alpha-tocopherol)_mgs,Vitamin B12_mcgs,Vitamin B6_mgs,Vitamin A (RAE)_mcgs,Net Carbohydrates_gms,Added Sugars_gms,Potassium_mgs_per_serving,Phosphorus_mgs_per_serving,Calcium_mgs_per_serving,Iron_mgs_per_serving,Magnesium_mgs_per_serving,Sodium_mgs_per_serving,Zinc_mgs_per_serving,Total Fat_gms_per_serving,Niacin (B3)_mgs_per_serving,Monounsaturated Fat_gms_per_serving,Polyunsaturated Fat_gms_per_serving,Riboflavin (B2)_mgs_per_serving,Thiamin (B1)_mgs_per_serving,Vitamin C_mgs_per_serving,Vitamin D_mcgs_per_serving,Cholesterol_mgs_per_serving,Saturated Fat_gms_per_serving,FATRN_per_serving,Dietary Fiber_gms_per_serving,Folic Acid_mcgs_per_serving,"Folate, food_mcgs_per_serving",Total Sugars_gms_per_serving,Vitamin K_mcgs_per_serving,Water_gms_per_serving,Carbohydrates_gms_per_serving,"Folate, DFE_mcgs_per_serving",Protein_gms_per_serving,Vitamin E (alpha-tocopherol)_mgs_per_serving,Vitamin B12_mcgs_per_serving,Vitamin B6_mgs_per_serving,Vitamin A (RAE)_mcgs_per_serving,Net Carbohydrates_gms_per_serving,diet_labels_clean
0,Classic Cabbage Slaw with Grandmother Shinn's ...,"[""Balanced""]","[""Vegetarian"",""Gluten-Free"",""Peanut-Free"",""Tre...",[],"[""american""]","[""lunch/dinner""]","[""salad""]","[{""food"":""kosher salt"",""text"":""1 tablespoon ko...",85.0,206.0,1344.937283,378.238500,316.216893,4.986130,93.095613,2866.738895,2.631570,15.262045,2.399385,5.073339,2.436278,0.750850,0.346041,124.065000,2.000000,386.160000,5.740132,12.633475,0.0,241.737750,55.653280,385.811000,1111.919491,74.524150,241.737750,19.597500,2.835580,0.940400,0.746793,1260.135250,61.890675,0.0,224.156214,63.039750,52.702815,0.831022,15.515935,477.789816,0.438595,2.543674,0.399898,0.845557,0.406046,0.125142,0.057673,20.677500,0.333333,64.360000,0.956689,0.0,2.105579,0.0,40.289625,9.275547,64.301833,185.319915,12.420692,40.289625,3.266250,0.472597,0.156733,0.124466,210.022542,10.315112,"[, "", B, a, l, a, n, c, e, d, "", ]"
1,Black Bean Soup,"[""High-Fiber""]","[""Dairy-Free"",""Gluten-Free"",""Egg-Free"",""Peanut...","[""Sulfites""]","[""american""]","[""lunch/dinner""]","[""soup""]","[{""food"":""ham steak"",""text"":""1 pound fully coo...",231.0,417.0,5951.657618,2361.677645,572.269647,23.629833,712.189757,12553.539068,20.479998,55.539721,22.703771,27.547140,7.578004,1.925726,4.132669,79.460838,4.082331,317.514659,15.555026,56.502798,0.0,1271.057411,11.406618,1144.806458,2871.681702,158.914882,1271.057411,181.811980,12.803019,3.991613,3.054568,1519.692579,102.412084,NaN,743.957202,295.209706,71.533706,2.953729,89.023720,1569.192384,2.560000,6.942465,2.837971,3.443393,0.947250,0.240716,0.516584,9.932605,0.510291,39.689332,1.944378,0.0,7.062850,0.0,158.882176,1.425827,143.100807,358.960213,19.864360,158.882176,22.726498,1.600377,0.498952,0.381821,189.961572,12.801510,"[, "", H, i, g, h, , F, i, b, e, r, "", ]"
2,Eat for Eight Bucks: Tofu with Tomatoes and Ci...,"[""High-Fiber"",""Low-Carb""]","[""Vegan"",""Vegetarian"",""Dairy-Free"",""Egg-Free"",...","[""Gluten"",""Wheat"",""Sulfites""]","[""asian""]","[""lunch/dinner""]","[""main course""]","[{""food"":""firm tofu"",""text"":""1 pound medium to...",410.0,363.0,3055.594090,1334.753701,3238.233026,19.513043,413.247690,2006.309099,10.884267,102.616128,15.456769,46.301237,40.946272,1.398109,1.630974,91.623990,0.293323,0.000000,10.899882,24.691537,84.0,318.857964,0.000000,0.000000,1089.618589,103.412350,461.357964,101.236576,0.000000,0.058665,1.305936,583.410517,78.720

In [ ]:

df["diet_labels_clean"] = df["diet_labels_clean"].str.replace(r'[\[\]"]', '', regex=True)


df["diet_labels_clean"] = df["diet_labels_clean"].str.strip()




In [ ]:
df['diet_labels'] = (
    df['diet_labels']
      .str.strip('[]')       
      .str.replace('"', '', regex=False)  
      .str.replace("'", "", regex=False)  
)


In [43]:
df = df.drop("diet_labels_clean", axis = 1)

In [ ]:

df["health_labels"] = df["health_labels"].str.replace(r'[\[\]"]', '', regex=True)

df["health_labels"] = df["health_labels"].str.strip()

df['health_labels'] = (
    df['health_labels']
      .str.strip('[]')        
      .str.replace('"', '', regex=False)  
      .str.replace("'", "", regex=False)  
)



In [46]:
df["diet_labels"].replace("", None, inplace=True)


In [ ]:

df["cautions"] = df["cautions"].str.replace(r'[\[\]"]', '', regex=True)


df["cautions"] = df["cautions"].str.strip()

df['cautions'] = (
    df['cautions']
      .str.strip('[]')       
      .str.replace('"', '', regex=False)  
      .str.replace("'", "", regex=False)  
)



In [48]:
df["cautions"].replace("", None, inplace=True)


In [ ]:

df["cuisine_type"] = df["cuisine_type"].str.replace(r'[\[\]"]', '', regex=True)


df["cuisine_type"] = df["cuisine_type"].str.strip()

df['cuisine_types'] = (
    df['cuisine_type']
      .str.strip('[]')        
      .str.replace('"', '', regex=False) 
      .str.replace("'", "", regex=False)  
)

df["cuisine_type"].replace("", None, inplace=True)


In [ ]:

df["dish_type"] = df["dish_type"].str.replace(r'[\[\]"]', '', regex=True)


df["dish_type"] = df["dish_type"].str.strip()

df['dish_type'] = (
    df['dish_type']
      .str.strip('[]')      
      .str.replace('"', '', regex=False)  
      .str.replace("'", "", regex=False)  
)

df["dish_type"].replace("", None, inplace=True)


In [ ]:

df["meal_type"] = df["meal_type"].str.replace(r'[\[\]"]', '', regex=True)


df["meal_type"] = df["meal_type"].str.strip()

df['meal_type'] = (
    df['meal_type']
      .str.strip('[]')       
      .str.replace('"', '', regex=False)  
      .str.replace("'", "", regex=False) 
)

df["meal_type"].replace("", None, inplace=True)


In [56]:
cols_to_drop = [
    'Potassium_mgs', 'Phosphorus_mgs', 'Calcium_mgs', 'Iron_mgs', 'Magnesium_mgs',
    'Sodium_mgs', 'Zinc_mgs', 'Total Fat_gms', 'Niacin (B3)_mgs',
    'Monounsaturated Fat_gms', 'Polyunsaturated Fat_gms', 'Riboflavin (B2)_mgs',
    'Thiamin (B1)_mgs', 'Vitamin C_mgs', 'Vitamin D_mcgs', 'Cholesterol_mgs',
    'Saturated Fat_gms', 'Dietary Fiber_gms', 'Folic Acid_mcgs',
    'Folate, food_mcgs', 'Total Sugars_gms', 'Vitamin K_mcgs', 'Water_gms',
    'Carbohydrates_gms', 'Folate, DFE_mcgs', 'Protein_gms',
    'Vitamin E (alpha-tocopherol)_mgs', 'Vitamin B12_mcgs', 'Vitamin B6_mgs',
    'Vitamin A (RAE)_mcgs', 'Net Carbohydrates_gms', 'Added Sugars_gms'
]

df = df.drop(columns=cols_to_drop, errors='ignore')


In [59]:
df = df.drop("cuisine_types", axis = 1)

In [61]:
import ast

df["ingredients"] = df["ingredients"].apply(
    lambda x: ast.literal_eval(x.replace("null", "None"))
)


In [62]:
df["ingredients_only"] = df["ingredients"].apply(
    lambda items: ", ".join({item["food"] for item in items})
)


In [63]:
df.head()

,recipe_name,diet_labels,health_labels,cautions,cuisine_type,meal_type,dish_type,ingredients,calorie_count,serving_size_gms,Potassium_mgs_per_serving,Phosphorus_mgs_per_serving,Calcium_mgs_per_serving,Iron_mgs_per_serving,Magnesium_mgs_per_serving,Sodium_mgs_per_serving,Zinc_mgs_per_serving,Total Fat_gms_per_serving,Niacin (B3)_mgs_per_serving,Monounsaturated Fat_gms_per_serving,Polyunsaturated Fat_gms_per_serving,Riboflavin (B2)_mgs_per_serving,Thiamin (B1)_mgs_per_serving,Vitamin C_mgs_per_serving,Vitamin D_mcgs_per_serving,Cholesterol_mgs_per_serving,Saturated Fat_gms_per_serving,FATRN_per_serving,Dietary Fiber_gms_per_serving,Folic Acid_mcgs_per_serving,"Folate, food_mcgs_per_serving",Total Sugars_gms_per_serving,Vitamin K_mcgs_per_serving,Water_gms_per_serving,Carbohydrates_gms_per_serving,"Folate, DFE_mcgs_per_serving",Protein_gms_per_serving,Vitamin E (alpha-tocopherol)_mgs_per_serving,Vitamin B12_mcgs_per_serving,Vitamin B6_mgs_per_serving,Vitamin A (RAE)_mcgs_per_serving,Net Carbohydrates_gms_per_serving,ingredients_only
0,Classic Cabbage Slaw with Grandmother Shinn's ...,Balanced,"Vegetarian,Gluten-Free,Peanut-Free,Tree-Nut-Fr...",None,american,lunch/dinner,salad,"[{'food': 'kosher salt', 'text': '1 tablespoon...",85.0,206.0,224.156214,63.039750,52.702815,0.831022,15.515935,477.789816,0.438595,2.543674,0.399898,0.845557,0.406046,0.125142,0.057673,20.677500,0.333333,64.360000,0.956689,0.0,2.105579,0.0,40.289625,9.275547,64.301833,185.319915,12.420692,40.289625,3.266250,0.472597,0.156733,0.124466,210.022542,10.315112,"scallions, apple cider vinegar, dry mustard, g..."
1,Black Bean Soup,High-Fiber,"Dairy-Free,Gluten-Free,Egg-Free,Peanut-Free,Tr...",Sulfites,american,lunch/dinner,soup,"[{'food': 'ham steak', 'text': '1 pound fully ...",231.0,417.0,743.957202,295.209706,71.533706,2.953729,89.023720,1569.192384,2.560000,6.942465,2.837971,3.443393,0.947250,0.240716,0.516584,9.932605,0.510291,39.689332,1.944378,0.0,7.062850,0.0,158.882176,1.425827,143.100807,358.960213,19.864360,158.882176,22.726498,1.600377,0.498952,0.381821,189.961572,12.801510,"olive oil, black pepper, Salt, garlic, water, ..."
2,Eat for Eight Bucks: Tofu with Tomatoes and Ci...,"High-Fiber,Low-Carb","Vegan,Vegetarian,Dairy-Free,Egg-Free,Tree-Nut-...","Gluten,Wheat,Sulfites",asian,lunch/dinner,main course,"[{'food': 'firm tofu', 'text': '1 pound medium...",410.0,363.0,763.898522,333.688425,809.558256,4.878261,103.311922,501.577275,2.721067,25.654032,3.864192,11.575309,10.236568,0.349527,0.407743,22.905997,0.073331,0.000000,2.724970,0.0,6.172884,21.0,79.714491,0.000000,0.000000,272.404647,25.853087,115.339491,25.309144,0.000000,0.014666,0.326484,145.852629,19.680203,"scallions, cilantro, button mushrooms, toasted..."
3,Fried Chicken Banh Mi,High-Fiber,"Peanut-Free,Tree-Nut-Free,Fish-Free,Shellfish-...","Gluten,Wheat,Sulfites",south east asian,lunch/dinner,sandwiches,"[{'food': 'oil', 'text': 'Neutral oil, for fry...",2117.0,636.0,0.000000,0.000000,0.000000,0.000000,0.000000,1989.915250,0.000000,139.832311,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,208.411250,0.000000,0.0,8.180700,0.0,0.000000,0.000000,0.000000,0.000000,165.581695,0.000000,50.828072,0.000000,0.000000,0.000000,0.000000,157.400995,"oil, baguettes, jalapeño, panko breadcrumbs, m..."
4,The Macaron Frappé,None,"Vegetarian,Peanut-Free,Tree-Nut-Free,Soy-Free,...",Sulfites,french,snack,desserts,"[{'food': 'macaron', 'text': '1 macaron (about...",276.0,438.0,309.814957,181.418838,230.146385,0.212233,27.916563,148.884705,1.033935,14.428496,0.216324,3.179054,0.651036,0.373085,0.086015,0.396000,1.718000,41.240000,9.599727,0.0,1.184913,0.0,9.825243,26.559992,0.719922,385.723746,30.106954,9.825243,6.581078,0.310332,0.806400,0.089208,134.000000,28.922041,"ice, vanilla ice cream, milk, macaron"


In [64]:
df= df.drop("ingredients", axis =1)

In [65]:
df["ingredients"] = df["ingredients_only"]

In [66]:
df.head()

,recipe_name,diet_labels,health_labels,cautions,cuisine_type,meal_type,dish_type,calorie_count,serving_size_gms,Potassium_mgs_per_serving,Phosphorus_mgs_per_serving,Calcium_mgs_per_serving,Iron_mgs_per_serving,Magnesium_mgs_per_serving,Sodium_mgs_per_serving,Zinc_mgs_per_serving,Total Fat_gms_per_serving,Niacin (B3)_mgs_per_serving,Monounsaturated Fat_gms_per_serving,Polyunsaturated Fat_gms_per_serving,Riboflavin (B2)_mgs_per_serving,Thiamin (B1)_mgs_per_serving,Vitamin C_mgs_per_serving,Vitamin D_mcgs_per_serving,Cholesterol_mgs_per_serving,Saturated Fat_gms_per_serving,FATRN_per_serving,Dietary Fiber_gms_per_serving,Folic Acid_mcgs_per_serving,"Folate, food_mcgs_per_serving",Total Sugars_gms_per_serving,Vitamin K_mcgs_per_serving,Water_gms_per_serving,Carbohydrates_gms_per_serving,"Folate, DFE_mcgs_per_serving",Protein_gms_per_serving,Vitamin E (alpha-tocopherol)_mgs_per_serving,Vitamin B12_mcgs_per_serving,Vitamin B6_mgs_per_serving,Vitamin A (RAE)_mcgs_per_serving,Net Carbohydrates_gms_per_serving,ingredients_only,ingredients
0,Classic Cabbage Slaw with Grandmother Shinn's ...,Balanced,"Vegetarian,Gluten-Free,Peanut-Free,Tree-Nut-Fr...",None,american,lunch/dinner,salad,85.0,206.0,224.156214,63.039750,52.702815,0.831022,15.515935,477.789816,0.438595,2.543674,0.399898,0.845557,0.406046,0.125142,0.057673,20.677500,0.333333,64.360000,0.956689,0.0,2.105579,0.0,40.289625,9.275547,64.301833,185.319915,12.420692,40.289625,3.266250,0.472597,0.156733,0.124466,210.022542,10.315112,"scallions, apple cider vinegar, dry mustard, g...","scallions, apple cider vinegar, dry mustard, g..."
1,Black Bean Soup,High-Fiber,"Dairy-Free,Gluten-Free,Egg-Free,Peanut-Free,Tr...",Sulfites,american,lunch/dinner,soup,231.0,417.0,743.957202,295.209706,71.533706,2.953729,89.023720,1569.192384,2.560000,6.942465,2.837971,3.443393,0.947250,0.240716,0.516584,9.932605,0.510291,39.689332,1.944378,0.0,7.062850,0.0,158.882176,1.425827,143.100807,358.960213,19.864360,158.882176,22.726498,1.600377,0.498952,0.381821,189.961572,12.801510,"olive oil, black pepper, Salt, garlic, water, ...","olive oil, black pepper, Salt, garlic, water, ..."
2,Eat for Eight Bucks: Tofu with Tomatoes and Ci...,"High-Fiber,Low-Carb","Vegan,Vegetarian,Dairy-Free,Egg-Free,Tree-Nut-...","Gluten,Wheat,Sulfites",asian,lunch/dinner,main course,410.0,363.0,763.898522,333.688425,809.558256,4.878261,103.311922,501.577275,2.721067,25.654032,3.864192,11.575309,10.236568,0.349527,0.407743,22.905997,0.073331,0.000000,2.724970,0.0,6.172884,21.0,79.714491,0.000000,0.000000,272.404647,25.853087,115.339491,25.309144,0.000000,0.014666,0.326484,145.852629,19.680203,"scallions, cilantro, button mushrooms, toasted...","scallions, cilantro, button mushrooms, toasted..."
3,Fried Chicken Banh Mi,High-Fiber,"Peanut-Free,Tree-Nut-Free,Fish-Free,Shellfish-...","Gluten,Wheat,Sulfites",south east asian,lunch/dinner,sandwiches,2117.0,636.0,0.000000,0.000000,0.000000,0.000000,0.000000,1989.915250,0.000000,139.832311,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,208.411250,0.000000,0.0,8.180700,0.0,0.000000,0.000000,0.000000,0.000000,165.581695,0.000000,50.828072,0.000000,0.000000,0.000000,0.000000,157.400995,"oil, baguettes, jalapeño, panko breadcrumbs, m...","oil, baguettes, jalapeño, panko breadcrumbs, m..."
4,The Macaron Frappé,None,"Vegetarian,Peanut-Free,Tree-Nut-Free,Soy-Free,...",Sulfites,french,snack,desserts,276.0,438.0,309.814957,181.418838,230.146385,0.212233,27.916563,148.884705,1.033935,14.428496,0.216324,3.179054,0.651036,0.373085,0.086015,0.396000,1.718000,41.240000,9.599727,0.0,1.184913,0.0,9.825243,26.559992,0.719922,385.723746,30.106954,9.825243,6.581078,0.310332,0.806400,0.089208,134.000000,28.922041,"ice, vanilla ice cream, milk, macaron","ice, vanilla ice cream, milk, macaron"


In [67]:
df= df.drop("FATRN_per_serving", axis =1)

In [68]:
df.head()

,recipe_name,diet_labels,health_labels,cautions,cuisine_type,meal_type,dish_type,calorie_count,serving_size_gms,Potassium_mgs_per_serving,Phosphorus_mgs_per_serving,Calcium_mgs_per_serving,Iron_mgs_per_serving,Magnesium_mgs_per_serving,Sodium_mgs_per_serving,Zinc_mgs_per_serving,Total Fat_gms_per_serving,Niacin (B3)_mgs_per_serving,Monounsaturated Fat_gms_per_serving,Polyunsaturated Fat_gms_per_serving,Riboflavin (B2)_mgs_per_serving,Thiamin (B1)_mgs_per_serving,Vitamin C_mgs_per_serving,Vitamin D_mcgs_per_serving,Cholesterol_mgs_per_serving,Saturated Fat_gms_per_serving,Dietary Fiber_gms_per_serving,Folic Acid_mcgs_per_serving,"Folate, food_mcgs_per_serving",Total Sugars_gms_per_serving,Vitamin K_mcgs_per_serving,Water_gms_per_serving,Carbohydrates_gms_per_serving,"Folate, DFE_mcgs_per_serving",Protein_gms_per_serving,Vitamin E (alpha-tocopherol)_mgs_per_serving,Vitamin B12_mcgs_per_serving,Vitamin B6_mgs_per_serving,Vitamin A (RAE)_mcgs_per_serving,Net Carbohydrates_gms_per_serving,ingredients_only,ingredients
0,Classic Cabbage Slaw with Grandmother Shinn's ...,Balanced,"Vegetarian,Gluten-Free,Peanut-Free,Tree-Nut-Fr...",None,american,lunch/dinner,salad,85.0,206.0,224.156214,63.039750,52.702815,0.831022,15.515935,477.789816,0.438595,2.543674,0.399898,0.845557,0.406046,0.125142,0.057673,20.677500,0.333333,64.360000,0.956689,2.105579,0.0,40.289625,9.275547,64.301833,185.319915,12.420692,40.289625,3.266250,0.472597,0.156733,0.124466,210.022542,10.315112,"scallions, apple cider vinegar, dry mustard, g...","scallions, apple cider vinegar, dry mustard, g..."
1,Black Bean Soup,High-Fiber,"Dairy-Free,Gluten-Free,Egg-Free,Peanut-Free,Tr...",Sulfites,american,lunch/dinner,soup,231.0,417.0,743.957202,295.209706,71.533706,2.953729,89.023720,1569.192384,2.560000,6.942465,2.837971,3.443393,0.947250,0.240716,0.516584,9.932605,0.510291,39.689332,1.944378,7.062850,0.0,158.882176,1.425827,143.100807,358.960213,19.864360,158.882176,22.726498,1.600377,0.498952,0.381821,189.961572,12.801510,"olive oil, black pepper, Salt, garlic, water, ...","olive oil, black pepper, Salt, garlic, water, ..."
2,Eat for Eight Bucks: Tofu with Tomatoes and Ci...,"High-Fiber,Low-Carb","Vegan,Vegetarian,Dairy-Free,Egg-Free,Tree-Nut-...","Gluten,Wheat,Sulfites",asian,lunch/dinner,main course,410.0,363.0,763.898522,333.688425,809.558256,4.878261,103.311922,501.577275,2.721067,25.654032,3.864192,11.575309,10.236568,0.349527,0.407743,22.905997,0.073331,0.000000,2.724970,6.172884,21.0,79.714491,0.000000,0.000000,272.404647,25.853087,115.339491,25.309144,0.000000,0.014666,0.326484,145.852629,19.680203,"scallions, cilantro, button mushrooms, toasted...","scallions, cilantro, button mushrooms, toasted..."
3,Fried Chicken Banh Mi,High-Fiber,"Peanut-Free,Tree-Nut-Free,Fish-Free,Shellfish-...","Gluten,Wheat,Sulfites",south east asian,lunch/dinner,sandwiches,2117.0,636.0,0.000000,0.000000,0.000000,0.000000,0.000000,1989.915250,0.000000,139.832311,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,208.411250,0.000000,8.180700,0.0,0.000000,0.000000,0.000000,0.000000,165.581695,0.000000,50.828072,0.000000,0.000000,0.000000,0.000000,157.400995,"oil, baguettes, jalapeño, panko breadcrumbs, m...","oil, baguettes, jalapeño, panko breadcrumbs, m..."
4,The Macaron Frappé,None,"Vegetarian,Peanut-Free,Tree-Nut-Free,Soy-Free,...",Sulfites,french,snack,desserts,276.0,438.0,309.814957,181.418838,230.146385,0.212233,27.916563,148.884705,1.033935,14.428496,0.216324,3.179054,0.651036,0.373085,0.086015,0.396000,1.718000,41.240000,9.599727,1.184913,0.0,9.825243,26.559992,0.719922,385.723746,30.106954,9.825243,6.581078,0.310332,0.806400,0.089208,134.000000,28.922041,"ice, vanilla ice cream, milk, macaron","ice, vanilla ice cream, milk, macaron"


In [69]:
df.to_csv("nutrition_processed.csv", index=False)


In [70]:
import numpy as np

df = df.replace(r"^\s*$", np.nan, regex=True)


In [71]:
df.to_csv("nutrition_final.csv", index=False)


In [72]:
df.head(10)

,recipe_name,diet_labels,health_labels,cautions,cuisine_type,meal_type,dish_type,calorie_count,serving_size_gms,Potassium_mgs_per_serving,Phosphorus_mgs_per_serving,Calcium_mgs_per_serving,Iron_mgs_per_serving,Magnesium_mgs_per_serving,Sodium_mgs_per_serving,Zinc_mgs_per_serving,Total Fat_gms_per_serving,Niacin (B3)_mgs_per_serving,Monounsaturated Fat_gms_per_serving,Polyunsaturated Fat_gms_per_serving,Riboflavin (B2)_mgs_per_serving,Thiamin (B1)_mgs_per_serving,Vitamin C_mgs_per_serving,Vitamin D_mcgs_per_serving,Cholesterol_mgs_per_serving,Saturated Fat_gms_per_serving,Dietary Fiber_gms_per_serving,Folic Acid_mcgs_per_serving,"Folate, food_mcgs_per_serving",Total Sugars_gms_per_serving,Vitamin K_mcgs_per_serving,Water_gms_per_serving,Carbohydrates_gms_per_serving,"Folate, DFE_mcgs_per_serving",Protein_gms_per_serving,Vitamin E (alpha-tocopherol)_mgs_per_serving,Vitamin B12_mcgs_per_serving,Vitamin B6_mgs_per_serving,Vitamin A (RAE)_mcgs_per_serving,Net Carbohydrates_gms_per_serving,ingredients_only,ingredients
0,Classic Cabbage Slaw with Grandmother Shinn's ...,Balanced,"Vegetarian,Gluten-Free,Peanut-Free,Tree-Nut-Fr...",None,american,lunch/dinner,salad,85.0,206.0,224.156214,63.039750,52.702815,0.831022,15.515935,477.789816,0.438595,2.543674,0.399898,0.845557,0.406046,0.125142,0.057673,20.677500,0.333333,64.360000,0.956689,2.105579,0.00000,40.289625,9.275547,64.301833,185.319915,12.420692,40.289625,3.266250,0.472597,0.156733,0.124466,210.022542,10.315112,"scallions, apple cider vinegar, dry mustard, g...","scallions, apple cider vinegar, dry mustard, g..."
1,Black Bean Soup,High-Fiber,"Dairy-Free,Gluten-Free,Egg-Free,Peanut-Free,Tr...",Sulfites,american,lunch/dinner,soup,231.0,417.0,743.957202,295.209706,71.533706,2.953729,89.023720,1569.192384,2.560000,6.942465,2.837971,3.443393,0.947250,0.240716,0.516584,9.932605,0.510291,39.689332,1.944378,7.062850,0.00000,158.882176,1.425827,143.100807,358.960213,19.864360,158.882176,22.726498,1.600377,0.498952,0.381821,189.961572,12.801510,"olive oil, black pepper, Salt, garlic, water, ...","olive oil, black pepper, Salt, garlic, water, ..."
2,Eat for Eight Bucks: Tofu with Tomatoes and Ci...,"High-Fiber,Low-Carb","Vegan,Vegetarian,Dairy-Free,Egg-Free,Tree-Nut-...","Gluten,Wheat,Sulfites",asian,lunch/dinner,main course,410.0,363.0,763.898522,333.688425,809.558256,4.878261,103.311922,501.577275,2.721067,25.654032,3.864192,11.575309,10.236568,0.349527,0.407743,22.905997,0.073331,0.000000,2.724970,6.172884,21.00000,79.714491,0.000000,0.000000,272.404647,25.853087,115.339491,25.309144,0.000000,0.014666,0.326484,145.852629,19.680203,"scallions, cilantro, button mushrooms, toasted...","scallions, cilantro, button mushrooms, toasted..."
3,Fried Chicken Banh Mi,High-Fiber,"Peanut-Free,Tree-Nut-Free,Fish-Free,Shellfish-...","Gluten,Wheat,Sulfites",south east asian,lunch/dinner,sandwiches,2117.0,636.0,0.000000,0.000000,0.000000,0.000000,0.000000,1989.915250,0.000000,139.832311,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,208.411250,0.000000,8.180700,0.00000,0.000000,0.000000,0.000000,0.000000,165.581695,0.000000,50.828072,0.000000,0.000000,0.000000,0.000000,157.400995,"oil, baguettes, jalapeño, panko breadcrumbs, m...","oil, baguettes, jalapeño, panko breadcrumbs, m..."
4,The Macaron Frappé,None,"Vegetarian,Peanut-Free,Tree-Nut-Free,Soy-Free,...",Sulfites,french,snack,desserts,276.0,438.0,309.814957,181.418838,230.146385,0.212233,27.916563,148.884705,1.033935,14.428496,0.216324,3.179054,0.651036,0.373085,0.086015,0.396000,1.718000,41.240000,9.599727,1.184913,0.00000,9.825243,26.559992,0.719922,385.723746,30.106954,9.825243,6.581078,0.310332,0.806400,0.089208,134.000000,28.922041,"ice, vanilla ice cream, milk, macaron","ice, vanilla ice cream, milk, macaron"
5,Cabbage au Gratin,Low-Carb,"Vegetarian,Gluten-Free,Egg-Free,Peanut-Free,Tr...",Sulfites,french,lunch/dinner,main course,339.0,176.0,223.695935,229.390656,378.368772,2.338567,29.085519,408.882044,1.065672,26.476310,0.343472,

In [73]:
df.to_csv("nutrition_processed.csv", index=False)


In [74]:
df = df.drop("ingredients_only", axis = 1)

In [75]:
df.to_csv("nutrition_recipe_processed.csv", index=False)
